# Solución al problema de las 8 reinas con algoritmos genéticos mediante diferentes representaciones 

### Daniel Vallejo Aldana // Optimización Estocástca // CIMAT 2023

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from itertools import permutations
from typing import Optional
from tqdm import tqdm

Para el problema de las 8 reinas creamos un algoritmo genético en el cual creamos dos tipos de representaciones, la representación binaria que consiste en un vector de 24 entradas donde cada bloque de 3 bits representa la columna en la que se encuentra la reina en el renglon j. La otra representación consiste en una representación alfabética. La función de fitness en este caso es de maximización del score en donde se checa que no haya reinas dentro del mismo renglón, de la misma columna o que puedan atacarse en algún otro movimiento. Se considera que el problema está resuelto cuando se llega a 28.0

## Creación de las diferentes representaciones del problema de las 8 reinas

In [2]:
def create_random_binary_representation():
    empty=np.zeros(8*3,dtype=int)
    for i in range(8):
        position=random.randint(0,7)
        binrep=str(bin(position))[2:]
        for j in range(len(binrep)):
            empty[3*i+(2-j)]=int(binrep[j])
    return empty

def create_numerical_representation():
    empty=np.zeros(8,dtype=int)
    for i in range(8):
        empty[i]=random.randint(0,7) #Creamos una representación de los números
    return empty
def binary_rep_to_coordinates(binrep:np.array):
    '''
    En este caso las parejas representan el renglón y la columna asociada por lo que la coordenada
    (i,j) representa la reina situada en el renglón i, columna j
    '''
    pairs=[]
    num=0
    for i in range(8):
        num=0
        for j in range(3):
            num+=binrep[3*i+j]*2**j 
        pairs.append((i,num))
    return pairs

def numerical_to_coordinates(numrep:np.array):
    pairs=[]
    for i in range(numrep.shape[0]):
        pairs.append((i,numrep[i]))
    return pairs  

def fitness_socore(positions):
    score=0
    for p in positions:
        current_row,current_col=p 
        for new_p in positions:
            new_row,new_col=new_p
            if (new_row==current_row) and (new_col==current_col):
                continue
            if new_col==current_col:
                continue
            if new_row+new_col==current_row+current_col:
                continue
            if new_row-new_col==current_row-current_col:
                continue
            score+=1
    return score/2

def binary_fitness_score(binrep:np.array):
    positions=binary_rep_to_coordinates(binrep)
    return fitness_socore(positions)

def numerical_fitness_score(numrep:np.array):
    positions=numerical_to_coordinates(numrep)
    return fitness_socore(positions)

def create_random_binary_population(pop_size:int=10):
    init_pop=[]
    for i in range(pop_size):
        init_pop.append(create_random_binary_representation())
    return init_pop

def create_random_numerical_population(pop_size:int=10):
    init_pop=[]
    for i in range(pop_size):
        init_pop.append(create_numerical_representation())
    return init_pop

def select_best_candidates(population:list):
    parents=[]
    for ind in population:
        score_ratio=(binary_fitness_score(ind))/28.0 
        nmr=random.random()
        if nmr<score_ratio:
            parents.append(ind)
    return parents

def select_best_numerical_candidate(population:list):
    parents=[]
    for ind in population:
        score_ratio=numerical_fitness_score(ind)/28.0
        nmr=random.random()
        if nmr<score_ratio:
            parents.append(ind)
    return parents

def binary_crossover_points(parents:list,cross_points:Optional[int]=2):
    '''
    Operador de cruza en el caso binario, en este caso si definimos que el punto de cruza
    es None entonces se hace cruza uniforme, en otro caso se hacen dos puntos de cruza o 
    un solo punto de cruza
    '''
    points=[]
    if cross_points is not None:
        points=sorted(random.sample(range(parents[0].shape[0]),cross_points))
    offsprings=[]
    Perm=list(permutations(parents,2))
    for i in range(len(Perm)):
        if cross_points is not None:
            if len(points)==1:
                tmp_array=np.zeros_like(Perm[i][0])
                tmp_array[:points[0]]=Perm[i][0][:points[0]]
                tmp_array[points[0]:]=Perm[i][1][points[0]:]
                Perm[i][0][:points[0]]=Perm[i][1][:points[0]]
                offsprings.append(tmp_array)
                offsprings.append(Perm[i][0])
            if len(points)==2:
                tmp_array=np.zeros_like(Perm[i][0])
                tmp_array[:points[0]]=Perm[i][0][:points[0]]
                tmp_array[points[0]:points[1]]=Perm[i][1][points[0]:points[1]]
                tmp_array[points[1]:]=Perm[i][0][points[1]:]
                Perm[i][0][points[0]:points[1]]=Perm[i][1][points[0]:points[1]]
                offsprings.append(tmp_array)
                offsprings.append(Perm[i][0])
        else:
            #Hacemos una cruza uniforme de los individuos de la población
            tmp_array=np.zeros_like(Perm[i][0])
            for k in range(tmp_array.shape[0]):
                num=random.random()
                if num<=0.5:
                    tmp_array[k]=Perm[i][0][k]
                    Perm[i][0][k]=Perm[i][1][k]
                else:
                    tmp_array[k]=Perm[i][1][k]
            offsprings.append(tmp_array)
            offsprings.append(Perm[i][0])
    #Hay que hacer elementos únicos de los offsprings para la selección 
    prev=len(offsprings)
    offsprings=list(set(tuple(i) for i in offsprings))
    for i in range(len(offsprings)):
        offsprings[i]=np.array(offsprings[i])
    now=len(offsprings)
    return offsprings

def mutate_binary_individual(binrep:np.array):
    mutation_prob=1.0/binrep.shape[0]
    for i in range(binrep.shape[0]):
        num=random.random()
        if num<mutation_prob:
            binrep[i]=1-binrep[i] #Para el caso binario cambiamos un 0 por un 1 en la mutación
    return binrep

def mutate_numerical_inidivdual(numrep:np.array):
    mutation_prob=1/numrep.shape[0]
    for i in range(numrep.shape[0]):
        num=random.random()
        if num<mutation_prob:
            numrep[i]=(numrep[i]+1) % 8
    return numrep 
def mutate_binary_population(population:list):
    for i in range(len(population)):
        population[i]=mutate_binary_individual(population[i])
    return population

def mutate_numerical_population(population:list):
    for i in range(len(population)):
        population[i]=mutate_numerical_inidivdual(population[i])
    return population

def evolution_binary_population(pop:list,population_size:int=10,cross_points:int=1):
    crossover=binary_crossover_points(pop,cross_points=cross_points)
    crossover=select_best_candidates(crossover)
    mutation=mutate_binary_population(crossover)
    new_gen = sorted(mutation, key=lambda ind: binary_fitness_score(ind), reverse=True)[:population_size]
    random.shuffle(new_gen)
    scores=[]
    for ind in new_gen:
        scores.append(binary_fitness_score(ind))
    return new_gen,max(scores)

def evolution_numerical_population(pop:list,population_size:int=10,cross_points:int=1):
    crossover=binary_crossover_points(pop,cross_points=cross_points)
    crossover=select_best_numerical_candidate(crossover)
    mutation=mutate_numerical_population(crossover)
    new_gen = sorted(mutation, key=lambda ind: numerical_fitness_score(ind), reverse=True)[:population_size]
    random.shuffle(new_gen)
    scores=[]
    for ind in new_gen:
        scores.append(numerical_fitness_score(ind))
    return new_gen,max(scores)




# Ejecuciones con representación binaria

### 1 Punto de cruza

In [3]:
evaluations=[]
for i in range(30):
    counter=0
    score=0
    poblacion=create_random_binary_population(pop_size=50)
    while score<28.0:
        poblacion,score=evolution_binary_population(poblacion,cross_points=1,population_size=50)
        counter+=1
    print("Run: {} -- Generations Required: {}".format(i+1,counter))
    evaluations.append(counter)

Run: 1 -- Generations Required: 225
Run: 2 -- Generations Required: 24
Run: 3 -- Generations Required: 45
Run: 4 -- Generations Required: 410
Run: 5 -- Generations Required: 423
Run: 6 -- Generations Required: 203
Run: 7 -- Generations Required: 95
Run: 8 -- Generations Required: 25
Run: 9 -- Generations Required: 171
Run: 10 -- Generations Required: 60
Run: 11 -- Generations Required: 75
Run: 12 -- Generations Required: 161
Run: 13 -- Generations Required: 225
Run: 14 -- Generations Required: 57
Run: 15 -- Generations Required: 26
Run: 16 -- Generations Required: 126
Run: 17 -- Generations Required: 523
Run: 18 -- Generations Required: 59
Run: 19 -- Generations Required: 103
Run: 20 -- Generations Required: 21
Run: 21 -- Generations Required: 39
Run: 22 -- Generations Required: 58
Run: 23 -- Generations Required: 48
Run: 24 -- Generations Required: 96
Run: 25 -- Generations Required: 62
Run: 26 -- Generations Required: 124
Run: 27 -- Generations Required: 351
Run: 28 -- Generations Re

In [4]:
print("Max: {} -- Min: {} -- Mean: {} -- SD: {}".format(np.max(evaluations),np.min(evaluations),round(np.mean(evaluations),2),round(np.std(evaluations),2)))

Max: 523 -- Min: 21 -- Mean: 145.53 -- SD: 130.8


### 2 Puntos de cruza

In [5]:
evaluations=[]
for i in range(30):
    counter=0
    score=0
    poblacion=create_random_binary_population(pop_size=50)
    while score<28.0:
        poblacion,score=evolution_binary_population(poblacion,cross_points=2,population_size=50)
        counter+=1
    print("Run: {} -- Generations Required: {}".format(i+1,counter))
    evaluations.append(counter)

Run: 1 -- Generations Required: 17
Run: 2 -- Generations Required: 13
Run: 3 -- Generations Required: 11
Run: 4 -- Generations Required: 15
Run: 5 -- Generations Required: 13
Run: 6 -- Generations Required: 12
Run: 7 -- Generations Required: 8
Run: 8 -- Generations Required: 11
Run: 9 -- Generations Required: 8
Run: 10 -- Generations Required: 28
Run: 11 -- Generations Required: 8
Run: 12 -- Generations Required: 8
Run: 13 -- Generations Required: 17
Run: 14 -- Generations Required: 9
Run: 15 -- Generations Required: 20
Run: 16 -- Generations Required: 7
Run: 17 -- Generations Required: 7
Run: 18 -- Generations Required: 12
Run: 19 -- Generations Required: 2
Run: 20 -- Generations Required: 10
Run: 21 -- Generations Required: 13
Run: 22 -- Generations Required: 17
Run: 23 -- Generations Required: 20
Run: 24 -- Generations Required: 8
Run: 25 -- Generations Required: 16
Run: 26 -- Generations Required: 16
Run: 27 -- Generations Required: 9
Run: 28 -- Generations Required: 24
Run: 29 -- 

In [6]:
print("Max: {} -- Min: {} -- Mean: {} -- SD: {}".format(np.max(evaluations),np.min(evaluations),round(np.mean(evaluations),2),round(np.std(evaluations),2)))

Max: 28 -- Min: 2 -- Mean: 12.47 -- SD: 5.6


### Cruza Uniforme

In [7]:
evaluations=[]
for i in range(30):
    counter=0
    score=0
    poblacion=create_random_binary_population(pop_size=50)
    while (score<28.0) and (counter<10000):
        poblacion,score=evolution_binary_population(poblacion,cross_points=None,population_size=50)
        counter+=1
    print("Run: {} -- Generations Required: {}".format(i+1,counter))
    evaluations.append(counter)

Run: 1 -- Generations Required: 11
Run: 2 -- Generations Required: 4
Run: 3 -- Generations Required: 5
Run: 4 -- Generations Required: 6
Run: 5 -- Generations Required: 5
Run: 6 -- Generations Required: 9
Run: 7 -- Generations Required: 5
Run: 8 -- Generations Required: 5
Run: 9 -- Generations Required: 8
Run: 10 -- Generations Required: 22
Run: 11 -- Generations Required: 4
Run: 12 -- Generations Required: 10
Run: 13 -- Generations Required: 5
Run: 14 -- Generations Required: 7
Run: 15 -- Generations Required: 6
Run: 16 -- Generations Required: 14
Run: 17 -- Generations Required: 13
Run: 18 -- Generations Required: 7
Run: 19 -- Generations Required: 5
Run: 20 -- Generations Required: 3
Run: 21 -- Generations Required: 4
Run: 22 -- Generations Required: 5
Run: 23 -- Generations Required: 3
Run: 24 -- Generations Required: 5
Run: 25 -- Generations Required: 6
Run: 26 -- Generations Required: 7
Run: 27 -- Generations Required: 10
Run: 28 -- Generations Required: 8
Run: 29 -- Generations 

In [8]:
print("Max: {} -- Min: {} -- Mean: {} -- SD: {}".format(np.max(evaluations),np.min(evaluations),round(np.mean(evaluations),2),round(np.std(evaluations),2)))

Max: 22 -- Min: 3 -- Mean: 7.17 -- SD: 3.9


# Ejecuciones con codificación de un alfabeto de 8 caracteres 

### Cruza de un 1-punto

In [9]:
#Hacemos una cruza de un solo punto en lugar de la codificación binaria
evaluations=[]
for i in range(30):
    counter=0
    score=0
    poblacion=create_random_numerical_population(pop_size=50)
    while score<28.0:
        poblacion,score=evolution_numerical_population(poblacion,cross_points=1,population_size=50)
        counter+=1
    print("Run: {} -- Generations Required: {}".format(i+1,counter))
    evaluations.append(counter)

Run: 1 -- Generations Required: 160
Run: 2 -- Generations Required: 45
Run: 3 -- Generations Required: 31
Run: 4 -- Generations Required: 83
Run: 5 -- Generations Required: 88
Run: 6 -- Generations Required: 248
Run: 7 -- Generations Required: 603
Run: 8 -- Generations Required: 754
Run: 9 -- Generations Required: 77
Run: 10 -- Generations Required: 171
Run: 11 -- Generations Required: 536
Run: 12 -- Generations Required: 866
Run: 13 -- Generations Required: 11
Run: 14 -- Generations Required: 421
Run: 15 -- Generations Required: 566
Run: 16 -- Generations Required: 215
Run: 17 -- Generations Required: 254
Run: 18 -- Generations Required: 96
Run: 19 -- Generations Required: 10
Run: 20 -- Generations Required: 315
Run: 21 -- Generations Required: 48
Run: 22 -- Generations Required: 34
Run: 23 -- Generations Required: 174
Run: 24 -- Generations Required: 22
Run: 25 -- Generations Required: 221
Run: 26 -- Generations Required: 43
Run: 27 -- Generations Required: 158
Run: 28 -- Generations

In [10]:
print("Max: {} -- Min: {} -- Mean: {} -- SD: {}".format(np.max(evaluations),np.min(evaluations),round(np.mean(evaluations),2),round(np.std(evaluations),2)))

Max: 866 -- Min: 10 -- Mean: 244.1 -- SD: 236.97


### Cruza de 2-puntos

In [11]:
#Hacemos una cruza de un solo punto en lugar de la codificación binaria
evaluations=[]
for i in range(30):
    counter=0
    score=0
    poblacion=create_random_numerical_population(pop_size=50)
    while score<28.0:
        poblacion,score=evolution_numerical_population(poblacion,cross_points=2,population_size=50)
        counter+=1
    print("Run: {} -- Generations Required: {}".format(i+1,counter))
    evaluations.append(counter)

Run: 1 -- Generations Required: 11
Run: 2 -- Generations Required: 4
Run: 3 -- Generations Required: 3
Run: 4 -- Generations Required: 14
Run: 5 -- Generations Required: 10
Run: 6 -- Generations Required: 3
Run: 7 -- Generations Required: 12
Run: 8 -- Generations Required: 3
Run: 9 -- Generations Required: 20
Run: 10 -- Generations Required: 3
Run: 11 -- Generations Required: 7
Run: 12 -- Generations Required: 14
Run: 13 -- Generations Required: 15
Run: 14 -- Generations Required: 3
Run: 15 -- Generations Required: 6
Run: 16 -- Generations Required: 4
Run: 17 -- Generations Required: 13
Run: 18 -- Generations Required: 13
Run: 19 -- Generations Required: 9
Run: 20 -- Generations Required: 10
Run: 21 -- Generations Required: 13
Run: 22 -- Generations Required: 13
Run: 23 -- Generations Required: 26
Run: 24 -- Generations Required: 5
Run: 25 -- Generations Required: 13
Run: 26 -- Generations Required: 27
Run: 27 -- Generations Required: 12
Run: 28 -- Generations Required: 4
Run: 29 -- Ge

In [12]:
print("Max: {} -- Min: {} -- Mean: {} -- SD: {}".format(np.max(evaluations),np.min(evaluations),round(np.mean(evaluations),2),round(np.std(evaluations),2)))

Max: 27 -- Min: 3 -- Mean: 9.9 -- SD: 6.44


### Cruza Uniforme

In [13]:
#Hacemos una cruza de un solo punto en lugar de la codificación binaria
evaluations=[]
for i in range(30):
    counter=0
    score=0
    poblacion=create_random_numerical_population(pop_size=50)
    while (score<28.0) and (counter<10000):
        poblacion,score=evolution_numerical_population(poblacion,cross_points=None,population_size=50)
        counter+=1
    print("Run: {} -- Generations Required: {}".format(i+1,counter))
    evaluations.append(counter)

Run: 1 -- Generations Required: 2
Run: 2 -- Generations Required: 2
Run: 3 -- Generations Required: 6
Run: 4 -- Generations Required: 5
Run: 5 -- Generations Required: 5
Run: 6 -- Generations Required: 9
Run: 7 -- Generations Required: 4
Run: 8 -- Generations Required: 5
Run: 9 -- Generations Required: 6
Run: 10 -- Generations Required: 3
Run: 11 -- Generations Required: 3
Run: 12 -- Generations Required: 9
Run: 13 -- Generations Required: 6
Run: 14 -- Generations Required: 3
Run: 15 -- Generations Required: 9
Run: 16 -- Generations Required: 7
Run: 17 -- Generations Required: 8
Run: 18 -- Generations Required: 8
Run: 19 -- Generations Required: 35
Run: 20 -- Generations Required: 6
Run: 21 -- Generations Required: 48
Run: 22 -- Generations Required: 3
Run: 23 -- Generations Required: 10
Run: 24 -- Generations Required: 4
Run: 25 -- Generations Required: 3
Run: 26 -- Generations Required: 5
Run: 27 -- Generations Required: 6
Run: 28 -- Generations Required: 4
Run: 29 -- Generations Req

In [14]:
print("Max: {} -- Min: {} -- Mean: {} -- SD: {}".format(np.max(evaluations),np.min(evaluations),round(np.mean(evaluations),2),round(np.std(evaluations),2)))

Max: 48 -- Min: 2 -- Mean: 7.77 -- SD: 9.44


### Tabla comparativa de los resultados

Con base en los resultados obtenidos en el presente trabajo podemos ver lo siguiente

|Representación|Tipo de Cruza|Max gen|Min Gen|Mean Gen|Std gen|
|---|---|---|---|---|---|
|Binaria|1-Punto|523|21|145.53|130.8|
|Binaria|2-Puntos|28|2|12.47|5.6|
|Binaria|Uniforme|22|3|7.17|3.9|
|Alfabética|1-punto|866|10|244.1|236.97|
|Alfabética|2-puntos|27|3|9.9|6.44|
|Alfabética|Uniforme|48|2|7.77|9.44|

De la tabla anterior podemos ver que la representación binaria con cruza uniforme fue la configuración que dió mejores resultados de los probados en el presente trabajo. Así mismo vemos que la cruza de 1 punto fue la que peores resultados obtuvo en el presente trabajo. En general, la cruza de dos puntos tiene igual muy buenos resultados. 